In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time
!pip install torchsummary

from torchsummary import summary
from tqdm import tqdm
import tensorflow
import tensorflow as tf
from sklearn.metrics import accuracy_score
from torchvision import models
import pickle
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

# MLP with three hidden layers and 800 nodes in each layer with batch normalization between every layer, and it’s activation in all hidden layers. ReLU and ELU are used as activations for NN and IOC respectively, and softmax is used in the last layer. We use Adam optimizer.

class MLP(nn.Module ):
    def __init__(self , IOCflag = False):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784*3, 800)
        self.fc2 = nn.Linear(800, 800)
        self.fc3 = nn.Linear(800, 800)
        self.fc4 = nn.Linear(800, 10)
        self.bn1 = nn.BatchNorm1d(800)
        self.bn2 = nn.BatchNorm1d(800)  
        self.bn3 = nn.BatchNorm1d(800)
        self.IOCflag = IOCflag
        self.activation = nn.ReLU()
        if self.IOCflag:
            self.activation = nn.ELU()
            

    def forward(self, x):
        x = x.view(-1, 3*784)
        x = self.bn1(self.activation(self.fc1(x)))
        x = self.bn2(self.activation(self.fc2(x)))
        x = self.bn3(self.activation(self.fc3(x)))
        x = F.softmax(self.fc4(x), dim=1)
        # x = nn.fc4(self.activation(x))
        # x = self.fc4(x)
        
        return x
    

    

In [16]:
# CIFAR10 dataset
transform = transforms.Compose([transforms.ToTensor(),transforms.CenterCrop(28),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=128,shuffle=True,num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=128,shuffle=False,num_workers=2)



Files already downloaded and verified
Files already downloaded and verified


In [4]:
# CIFAR10 dataset
transform = transforms.Compose([transforms.ToTensor(),transforms.CenterCrop(28),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform)

# for i in range(len(train_dataset)):
#         train_dataset.data[i] = tf.image.per_image_standardization(train_dataset.data[i])
# for i in range(len(test_dataset)):
#         test_dataset.data[i] = tf.image.per_image_standardization(test_dataset.data[i])

# Data loader
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64, shuffle=True, num_workers=2,pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=64, shuffle=False, num_workers=2,pin_memory=True)

print('Number of training examples: ', train_loader.dataset.data.shape)
print('Number of testing examples: ', test_loader.dataset.data.shape)
print(len(train_loader))


Files already downloaded and verified
Number of training examples:  (50000, 32, 32, 3)
Number of testing examples:  (10000, 32, 32, 3)
782


In [5]:
from tqdm import tqdm
pbar = tqdm(total=len(train_loader))
for idx, (data,label) in enumerate(train_loader):
    print(idx,data.size(),label.size())
    pbar.update(1)
    break
pbar.refresh()

  0%|          | 1/782 [00:08<1:45:18,  8.09s/it]

0 torch.Size([64, 3, 28, 28]) torch.Size([64])


True

In [5]:

def LabelCorruption(percentage,myloader):
    labels = myloader.dataset.targets[:]
    data = myloader.dataset.data
    num = int(len(labels)*percentage)
    idxes = []
    if percentage == 1:
        idxes = list(range(len(labels)))
    while len(idxes) < num:
        idx = np.random.randint(0,len(labels))
        if idx not in idxes:
            idxes.append(idx)
    for idx in idxes:
        randomLabel = np.random.randint(0,10)
        while randomLabel == labels[idx]:
            randomLabel = np.random.randint(0,10)
        labels[idx] = randomLabel
    corrupted_dataset = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=transform)
    corrupted_dataset.data = data
    corrupted_dataset.targets = labels
    corrupted_loader = torch.utils.data.DataLoader(dataset=corrupted_dataset, batch_size=128, shuffle=True)
    return corrupted_loader



In [ ]:
corrupted_trainloader = LabelCorruption(1,train_loader)

In [ ]:
def chkcorruptionLabels(train_loader , corrupter_trainloder):
    noOfIncorrectLabels = 0
    for i, (images, labels) in enumerate(train_loader):
        for j in range(len(labels)):
            if labels[j] != corrupter_trainloder.dataset.targets[i*128+j]:
                noOfIncorrectLabels += 1
    return noOfIncorrectLabels

print(chkcorruptionLabels(train_loader,corrupted_trainloader))

In [59]:

def test(model , test_loader , criterion):
    model.eval()
    correct = 0
    running_loss = 0.0
    running_correct = 0.0
    total = 0
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader,0):
            # print(len(data))
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_correct += (predicted == labels).sum().item()
    print("Test Accuracy" , running_correct/total , "Test Loss" , running_loss/total)
    return running_correct/total , running_loss/total



def train(model , train_loader,test_loader,scheduler , optimizer , epochs , IOCflag = False):
    model.train()
    train_loss = []
    train_acc = []
    test_losslst = []
    test_acclst = []
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        correct = 0
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        for batch_idx, data in enumerate(train_loader,0):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            if IOCflag:
              with torch.no_grad():

                for layer in [model.fc2.weight, model.fc3.weight, model.fc4.weight]:
                      layer[layer < 0] = torch.exp(layer[layer < 0] - 5)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_correct += (predicted == labels).sum().item()
            if batch_idx % 100 == 0:
                print("Epoch" , epoch , "Batch" , batch_idx , "Loss" , running_loss/total , "Accuracy" , running_correct/total)
                train_loss.append(running_loss/total)
                train_acc.append(running_correct/total)
                test_acc , test_loss = test(model , test_loader , criterion)
                test_losslst.append(test_loss)
                test_acclst.append(test_acc)
    scheduler.step()
    
    return train_loss , train_acc,test_losslst,test_acclst
            





In [53]:
MLPmodel = MLP()
MLPmodel.cuda()
optimizer = optim.Adam(MLPmodel.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

MLPtrain_loss , MLPtrain_acc , MLPtest_loss , MLPtest_acc = train(MLPmodel , corrupted_trainloader,test_loader,scheduler , optimizer , 50)

Epoch 0 Batch 0 Loss 0.017996549606323242 Accuracy 0.09375
Test Accuracy 0.1031 Test Loss 0.018189794945716858
Epoch 0 Batch 100 Loss 0.01798630249456014 Accuracy 0.10419245049504951
Test Accuracy 0.0453 Test Loss 0.01825043845176697
Epoch 0 Batch 200 Loss 0.01798382109559294 Accuracy 0.10607120646766169
Test Accuracy 0.0388 Test Loss 0.01833628737926483
Epoch 0 Batch 300 Loss 0.01798177931296865 Accuracy 0.10600083056478406
Test Accuracy 0.0324 Test Loss 0.01839664614200592
Epoch 1 Batch 0 Loss 0.017918886616826057 Accuracy 0.1484375
Test Accuracy 0.0295 Test Loss 0.018424300384521485
Epoch 1 Batch 100 Loss 0.017967571476751036 Accuracy 0.11130878712871287
Test Accuracy 0.032 Test Loss 0.01845751130580902
Epoch 1 Batch 200 Loss 0.017967371158857843 Accuracy 0.11326181592039801
Test Accuracy 0.0303 Test Loss 0.01849134967327118
Epoch 1 Batch 300 Loss 0.017966568828916233 Accuracy 0.11238579734219269
Test Accuracy 0.0323 Test Loss 0.018483714890480042
Epoch 2 Batch 0 Loss 0.017996529117

In [60]:

IOCmodel = MLP(IOCflag=True).cuda()
optimizer = optim.Adam(IOCmodel.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)
Ioctrain_loss , Ioctrain_acc , Ioctest_loss , Ioctest_acc = train(IOCmodel , train_loader,corrupted_trainloader,scheduler , optimizer , 25 , IOCflag = True)

Epoch 0 Batch 0 Loss 0.017869671806693077 Accuracy 0.1875
Test Accuracy 0.0985 Test Loss 0.018016371388435364
Epoch 0 Batch 100 Loss 0.017766732224585986 Accuracy 0.1608910891089109
Test Accuracy 0.09252 Test Loss 0.01823573387145996
Epoch 0 Batch 200 Loss 0.017758767384646545 Accuracy 0.16215796019900497
Test Accuracy 0.09512 Test Loss 0.018169270610809325
Epoch 0 Batch 300 Loss 0.017769539351281137 Accuracy 0.16115552325581395
Test Accuracy 0.09238 Test Loss 0.018305841755867004
Epoch 1 Batch 0 Loss 0.01781630516052246 Accuracy 0.140625
Test Accuracy 0.08856 Test Loss 0.018287194395065307
Epoch 1 Batch 100 Loss 0.017520985335554226 Accuracy 0.19152227722772278
Test Accuracy 0.08932 Test Loss 0.01829835401058197
Epoch 1 Batch 200 Loss 0.017527151917714385 Accuracy 0.19640080845771143
Test Accuracy 0.08792 Test Loss 0.0183915215921402
Epoch 1 Batch 300 Loss 0.017507422956764895 Accuracy 0.2006592607973422
Test Accuracy 0.08904 Test Loss 0.018367607927322387
Epoch 2 Batch 0 Loss 0.01689

KeyboardInterrupt: ignored

In [ ]:
# plot the loss and accuracy curves

def plot_loss(train_loss , test_loss , name):
    plt.plot(train_loss , label = "Train Loss")
    plt.plot(test_loss , label = "Test Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss vs Epochs for " + name)
    plt.legend()
    plt.show()

def plot_acc(train_acc , test_acc , name):
    plt.plot(train_acc , label = "Train Accuracy")
    plt.plot(test_acc , label = "Test Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs Epochs for " + name)
    plt.legend()
    plt.show()


plot_loss(MLPtrain_loss , MLPtest_loss , "MLP")
plot_loss(Ioctrain_loss , Ioctest_loss , "MLP with IOC")



In [ ]:
plot_acc(MLPtrain_acc , MLPtest_acc , "MLP")
plot_acc(Ioctrain_acc , Ioctest_acc , "MLP with IOC")

In [ ]:

# save the model
path = "/content/drive/My Drive/"
torch.save(MLPmodel.state_dict(), path + 'MLPmodel.pt')
torch.save(IOCmodel.state_dict(), path + 'IOCmodel.pt')

# save the loss and accuracy
MLPdata = {"train_loss" : MLPtrain_loss , "train_acc" : MLPtrain_acc , "test_loss" : MLPtest_loss , "test_acc" : MLPtest_acc}
IOCdata = {"train_loss" : Ioctrain_loss , "train_acc" : Ioctrain_acc , "test_loss" : Ioctest_loss , "test_acc" : Ioctest_acc}
with open(path + 'MLPdata.json', 'w') as fp:
    json.dump(MLPdata, fp)
with open(path + 'IOCdata.json', 'w') as fp:
    json.dump(IOCdata, fp)
    